# 00. Load Packages

In [1]:
import os
import pcraster as pcr

import pcr_tool

# 01. Basic Inputs for Example Run

In [2]:
epsg = 32645 # EPSG for clone.map
epsg_check = 4326 # EPSG for checking projections

path_clone = os.path.join('..', 'examples', 'Trisuli', 'input', 'clone.map') # path to clone.map
path_dem = os.path.join('..', 'examples', 'Trisuli', 'input', 'dem.map') # path to dem.map

# 02. Create Clonemap

First, we need to set an EPSG to define a clonemap.

In [3]:
map_clone = pcr_tool.clonemap(epsg = epsg)

Let's create a clonemap based on the clonemap properties.

In [4]:
map_clone.create_clonemap(nrow = 749, ncol = 485, cellsize = 200, west = 290289, north = 3214899)
pcr.aguila(map_clone.clonemap)

# 03. Set DEM and its Modifications

Here, we assume that the DEM pcraster map should be created through the pcr_tool operations.

In [5]:
wf_dem = map_clone.set_dem(path_dem)
pcr.aguila(wf_dem)

Oftentimes, there is a need to fill the depressions in DEM.

In [6]:
# equivalent to pcraster.lddcreatemap
wf_dem_fill = map_clone.fill_dem(replace = True) # set 'replace' as True to update 'map_clone.dem'
pcr.aguila(wf_dem_fill)

Let's check the differences between the orginal and filled DEMs.

In [7]:
wf_dem_diff = wf_dem_fill - wf_dem
pcr.aguila(wf_dem_diff)

From now on, the operations below will be based on 'map_clone.dem'.

In [8]:
pcr.aguila(map_clone.dem)

# 04. Calculate Hydrologic Properties

First, we're going to calculate the slope property and compare with the SPHY-provided input.

In [9]:
# equivalent to pcraster.slope
wf_slope = map_clone.calculate_slope()
pcr.aguila(wf_slope)

wf_slope_ref = pcr.readmap(os.path.join('..', 'examples', 'Trisuli', 'input', 'slope.map'))
pcr.aguila(wf_slope_ref)

pcr.aguila(wf_slope - wf_slope_ref)

Second, the flow direction property.

In [10]:
# equivalent to pcraster.lddcreate
wf_flowdir = map_clone.calculate_flowdir()
pcr.aguila(wf_flowdir)

wf_flowdir_ref = pcr.readmap(os.path.join('..', 'examples', 'Trisuli', 'input', 'ldd.map'))
pcr.aguila(wf_flowdir_ref)

Let's assign outlet points to delineate catchments.

In [11]:
list_outlet = [
    [27.9807, 85.1817],
    [28.2089, 85.5489],
    ]
wf_outlet, list_outlet_epsg = map_clone.create_outlet(outlets = list_outlet, epsg = epsg_check)
pcr.aguila(wf_outlet)
list_outlet_epsg

wf_outlet_ref = pcr.readmap(os.path.join('..', 'examples', 'Trisuli', 'input', 'outlet.map'))
pcr.aguila(wf_outlet_ref)

Based on the assigned outlet points, catchments can be delineated. You will see the difference with the SPHY-provided input because the DEM and flow direction require requires additional treatements to correctly delineate catchment.

In [12]:
# equivalent to pcraster.subcatchment
wf_catchment = map_clone.create_subcatchment()
pcr.aguila(wf_catchment)

wf_catchment_ref = pcr.readmap(os.path.join('..', 'examples', 'Trisuli', 'input', 'catchment.map'))
pcr.aguila(wf_catchment_ref)

While this is not required for SPHY applications, but we can also check whether channels are aligned well as expected through streamorder and flow accumulation.

In [13]:
# equivalent to pcraster.streamorder
wf_streamorder = map_clone.create_streamorder()
pcr.aguila(wf_streamorder)

wf_channel_streamorder = pcr.ifthen(wf_streamorder >= 4, pcr.boolean(1))
pcr.aguila(wf_channel_streamorder)

In [14]:
# equivalent to pcraster.accuflux
wf_accuflux = map_clone.create_accuflux()
pcr.aguila(wf_accuflux)

wf_channel_accuflux = pcr.ifthen(wf_accuflux >= 100, pcr.boolean(1))
pcr.aguila(wf_channel_accuflux)

However, SPHY requires a latitude pcraster map to calculate extraterrestrial radiation.

In [15]:
wf_lat = map_clone.create_latitude(field_mv = wf_dem) # 'field_mv' is used to replicate a mask
pcr.aguila(wf_lat)

wf_lat_ref = pcr.readmap(os.path.join('..', 'examples', 'Trisuli', 'input', 'latitude.map'))
pcr.aguila(wf_lat_ref)

pcr.aguila(wf_lat - wf_lat_ref)

# 05. Generate Soil/Land/Vegetation Property Map

Here is an example to create a pcraster map of saturated hydraulic conductivity for a root zone from 'soil.map' using lookuptable, 'root_satk.tbl'.

In [16]:
path_table = os.path.join('..', 'examples', 'Trisuli', 'soil', 'root_satk.tbl')
path_soilmap = os.path.join('..', 'examples', 'Trisuli', 'soil', 'soil.map')
map_root_satk = pcr.lookupscalar(path_table, path_soilmap)

pcr.aguila(path_soilmap)
pcr.aguila(map_root_satk)